<a href="https://colab.research.google.com/github/haiyinyang929/Statistical-Language-Processing-A4/blob/main/A4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Assignment 4

## Honor Code
Course:        Statistical Language Processing - Summer 2024  
Assignment:   A4
Author(s):    Yip, Hiu Yan / Yang, Hai-Yin  
Honor Code:   We pledge that this program represents our own work,
and that we have not given or received unauthorized help
with this assignment.

 **Overall Organization/Presentation of the Notebook (Total: 4 pts)  
Your notebook should be well organized and have markdown cells with headers and descriptions
of what the code is doing.**

## Part 1 Notebook Setup

- DONE Add a Markdown cell containing the honor code with your name(s)  

- DONE? Install and import the packages you need.  
- DONE Mount Google Drive and create two directories, one to save training
 checkpoints and one to save models  
- DONE Set variable device, which you need later:  
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
!pip install google-colab
!pip install datasets transformers evaluate
!pip install torch
!pip install accelerate -U
!pip install transformers[torch]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 22.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 12.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 24.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.1 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency reso

In [ ]:
from google.colab import drive
from datasets import load_dataset
import torch
from transformers import AdamW, AutoTokenizer, AutoModelForSequenceClassification

In [ ]:
# Mount Google Drive and create two directories, one to save training checkpoints and one to save models
drive.mount('/content/gdrive', force_remount=True)

# a new directory for training checkpoints
training_checkpoint_dir = "slp2-2024-assignment4-checkpoint"
!mkdir /content/gdrive/My\ Drive/$training_checkpoint_dir

# a new directory for saving models
models_dir = "slp2-2024-assignment4-models"
!mkdir /content/gdrive/My\ Drive/$models_dir

# navigate to your drive directory
%cd /content/gdrive/My\ Drive/

Mounted at /content/gdrive
mkdir: cannot create directory ‘/content/gdrive/My Drive/slp2-2024-assignment4-models’: File exists
/content/gdrive/My Drive


In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
# set up variable device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

## Part 2 Prepare the Training Data - Tokenization

3. Prepare the Training Data  
(a) Load the emotion subset of the tweet dataset (1 pt)  
(b) Tokenize the entire dataset (2 pts)  
(c) Create small data splits for cpu training (2 pts)    
  
For dataset loading instructions, see the "Use this dataset" button in the right column of the HuggingFace page.  
Until you get all bugs out of your code, you should use a cpu:  
Runtime -> Change runtime type -> CPU  
**Also, you don’t need to use much training data just for debugging your code.**  
If device.type == "cpu", use only the first (50/10/10) of the tokenized train, dev, test splits.  

In [ ]:
# Load model directly
from transformers import AutoTokenizer

checkpoint = "distilbert/distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

ds = load_dataset("cardiffnlp/tweet_eval", "emotion")

train_data = ds["train"]
dev_data = ds["validation"]
test_data = ds["test"]

def tokenize_function(data):
    return tokenizer(data['text'], truncation=True)

tokenized_ds = ds.map(tokenize_function, batched=True)


# # This is new
# batch["labels"] = torch.tensor([1, 1])

print(ds)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

...

Generating validation split:   0%|          | 0/374 [00:00<?, ? examples/s]

Map:   0%|          | 0/3257 [00:00<?, ? examples/s]

Map:   0%|          | 0/1421 [00:00<?, ? examples/s]

Map:   0%|          | 0/374 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 3257
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 1421
    })
    validation: Dataset({
        features: ['text', 'label'],
        num_rows: 374
    })
})


In [ ]:
drive.mount('/content/gdrive')
!mkdir /content/gdrive/My\ Drive/training-sm

tokenized_train_sm = tokenized_ds["train"].select(range(50))
tokenized_train_sm.save_to_disk("/content/gdrive/My Drive/training-sm")
print("save1")

tokenized_dev_sm = tokenized_ds["validation"].select(range(10))
tokenized_dev_sm.save_to_disk("/content/gdrive/My Drive/training-sm")
print("save2")

tokenized_test_sm = tokenized_ds["test"].select(range(10))
tokenized_test_sm.save_to_disk("/content/gdrive/My Drive/training-sm")
print("save3")

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
mkdir: cannot create directory ‘/content/gdrive/My Drive/training-sm’: File exists


Saving the dataset (0/1 shards):   0%|          | 0/50 [00:00<?, ? examples/s]

save1


Saving the dataset (0/1 shards):   0%|          | 0/10 [00:00<?, ? examples/s]

save2


Saving the dataset (0/1 shards):   0%|          | 0/10 [00:00<?, ? examples/s]

save3


In [ ]:
from transformers import DataCollatorWithPadding
# pad all the examples to the length of the longest element when we batch elements together
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)


#for testing only, not use elsewhere
samples = tokenized_train_sm[:50]

samples = {k: v for k, v in samples.items() if k in ["input_ids"]}
print([len(x) for x in samples["input_ids"]])

batch = data_collator(samples)
{k: v.shape for k, v in batch.items()}

[26, 30, 25, 29, 17, 30, 23, 22, 22, 12, 29, 32, 33, 13, 20, 32, 36, 25, 34, 38, 22, 35, 25, 29, 12, 36, 7, 27, 25, 28, 22, 18, 24, 31, 20, 6, 35, 19, 17, 34, 37, 16, 33, 38, 37, 20, 17, 27, 35, 25]


{'input_ids': torch.Size([50, 38]), 'attention_mask': torch.Size([50, 38])}

## Part 3 Training configurations

### Load the pretrained distilbert-base-uncased model
- from training feature, get the labels list and create corresponding label-to-id dicts.  
- Instantiate a model from AutoModelForSequenceClassification

In [ ]:
from transformers import Trainer, TrainingArguments

labels = tokenized_ds["train"].features["label"].names
num_labels = len(labels)

label2id = {k: labels.index(k) for k in tokenized_ds["train"].features["label"].names}

id2label = {id: label for label, id in label2id.items()}

model = AutoModelForSequenceClassification.from_pretrained(checkpoint,
                                                           num_labels=num_labels,
                                                           id2label=id2label,
                                                           label2id=label2id)

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert/distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


### Evaluation Setup

define a `compute_metrics` function for the trainer object

In [ ]:
import evaluate
import numpy as np
from torch.utils.data import DataLoader

# Trainer object can precit the output, output is the logits and labels
# e.g. eval_preds = trainer.predict(tokenized_train_sm)

def compute_metrics(eval_preds):
    metric = evaluate.load("f1")
    logits = eval_preds.predictions
    labels = eval_preds.label_ids
    # get the argmax item along each output vector
    predictions = np.argmax(logits, axis=-1)
    print(predictions)
    return metric.compute(predictions=predictions, references=labels, average='macro')

### Construct Trainer object
- set up Training Arguments
- instantiate a trainer object with the required parameters
- Change the `train_dataset=tokenized_train_sm`,
    `eval_dataset=tokenized_dev_sm` to the real data set (currently it is small just for testing)

In [ ]:
import accelerate
# change the output_dir to models_dir
training_args = TrainingArguments(output_dir= models_dir,
                                  save_total_limit=2,
                                  load_best_model_at_end=True,
                                  eval_strategy="steps",
                                  save_strategy="steps",
                                  logging_strategy="steps", max_steps=500)

# load_best_model_at_end=True, automatically load the best model at the end of training.

tokenized_ds_train = tokenized_ds["train"]
tokenized_ds_dev = tokenized_ds["validation"]

# train the full dataset
trainer = Trainer(
    model,
    training_args,
    train_dataset=tokenized_ds_train,
    eval_dataset=tokenized_ds_dev,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

max_steps is given, it will override any value given in num_train_epochs


### Launch training run

Important: Move the model to the device before training:

model.to(device)

When you are ready to train with the full dataset, request a GPU:

**Runtime -> Change runtime type -> T4 GPU**

Training for the default 3 epochs will take roughly 2 minutes on a GPU.



In [ ]:
model.to(device)

DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
 

In [ ]:
trainer.train()

Step,Training Loss,Validation Loss,F1
500,0.675500,0.649019,0.730518


[0 0 0 0 0 0 3 3 2 0 1 0 2 3 3 3 0 3 3 0 3 1 2 3 3 3 3 1 0 0 0 1 3 3 1 1 1
 0 2 1 1 3 1 0 1 1 3 3 0 0 3 1 3 3 0 0 0 0 1 3 3 2 3 1 3 0 0 1 3 0 0 0 3 2
 1 0 1 0 1 0 3 0 0 1 0 0 3 0 3 3 0 1 1 1 0 1 3 2 3 1 2 3 1 1 3 3 0 0 1 3 1
 0 1 1 3 1 0 1 3 1 0 0 0 1 0 3 1 0 1 0 0 1 0 0 0 2 0 3 3 0 3 0 0 0 0 3 3 0
 3 1 1 3 0 3 2 1 3 2 3 0 0 1 0 1 1 0 0 1 1 0 3 0 1 0 0 1 0 1 1 0 0 2 0 0 2
 1 1 0 0 0 1 2 3 0 0 3 3 0 1 0 0 3 0 3 0 2 0 1 0 1 0 0 3 1 2 1 3 3 2 1 0 3
 3 3 1 0 0 3 3 3 0 3 0 0 0 3 0 3 0 1 0 3 3 0 3 3 1 1 3 3 1 0 3 0 1 3 3 1 1
 3 1 0 0 1 0 1 1 3 0 1 0 0 0 2 0 3 0 0 3 0 1 0 3 1 1 0 0 0 3 0 0 0 0 3 0 1
 3 0 3 0 0 1 0 1 0 3 1 0 0 0 3 1 3 0 0 3 0 3 3 0 3 3 1 1 0 0 0 0 3 0 0 1 0
 1 3 3 3 0 0 0 3 0 0 0 0 3 0 0 3 0 3 3 0 1 0 3 2 1 3 1 2 3 0 1 0 0 1 1 0 3
 0 0 0 0]


TrainOutput(global_step=500, training_loss=0.6755064086914062, metrics={'train_runtime': 49.6234, 'train_samples_per_second': 80.607, 'train_steps_per_second': 10.076, 'total_flos': 40273854122712.0, 'train_loss': 0.6755064086914062, 'epoch': 1.2254901960784315})

### NOT RELEVANT
#### Learning Rate Scheduler Setup
- for adjusting the learning rate over time according to a predefined schedule or strategy

In [ ]:
# from transformers import get_scheduler

# optimizer = AdamW(model.parameters(), lr=5e-5)

# num_epochs = 3

# num_training_steps = num_epochs * len(train_dataloader)
# lr_scheduler = get_scheduler(
#     "linear",
#     optimizer=optimizer,
#     num_warmup_steps=0,
#     num_training_steps=num_training_steps,
# )


# model.to(device)
# device

### NOT RELEVANT
set training by hand
Imported a progress bar to show the training steps

In [ ]:
# from tqdm.auto import tqdm

# progress_bar = tqdm(range(num_training_steps))

# model.train()
# for epoch in range(num_epochs):
#     for batch in train_dataloader:
#         batch = {k: v.to(device) for k, v in batch.items()}
#         outputs = model(**batch)
#         loss = outputs.loss
#         loss.backward()

#         optimizer.step()
#         lr_scheduler.step()
#         optimizer.zero_grad()
#         progress_bar.update(1)


### NOT RELEVANT Evaluation Setup (2)

In [ ]:
# metric = evaluate.load("f1")
# model.eval()
# for batch in dev_dataloader:
#     batch = {k: v.to(device) for k, v in batch.items()}
#     with torch.no_grad():
#         outputs = model(**batch)

#     logits = outputs.logits
#     predictions = torch.argmax(logits, dim=-1)
#     metric.add_batch(predictions=predictions, references=batch["labels"])

# metric.compute(average='macro')

In [ ]:
eval_results = trainer.evaluate()

[0 0 0 0 0 0 3 3 2 0 1 0 2 3 3 3 0 3 3 0 3 1 2 3 3 3 3 1 0 0 0 1 3 3 1 1 1
 0 2 1 1 3 1 0 1 1 3 3 0 0 3 1 3 3 0 0 0 0 1 3 3 2 3 1 3 0 0 1 3 0 0 0 3 2
 1 0 1 0 1 0 3 0 0 1 0 0 3 0 3 3 0 1 1 1 0 1 3 2 3 1 2 3 1 1 3 3 0 0 1 3 1
 0 1 1 3 1 0 1 3 1 0 0 0 1 0 3 1 0 1 0 0 1 0 0 0 2 0 3 3 0 3 0 0 0 0 3 3 0
 3 1 1 3 0 3 2 1 3 2 3 0 0 1 0 1 1 0 0 1 1 0 3 0 1 0 0 1 0 1 1 0 0 2 0 0 2
 1 1 0 0 0 1 2 3 0 0 3 3 0 1 0 0 3 0 3 0 2 0 1 0 1 0 0 3 1 2 1 3 3 2 1 0 3
 3 3 1 0 0 3 3 3 0 3 0 0 0 3 0 3 0 1 0 3 3 0 3 3 1 1 3 3 1 0 3 0 1 3 3 1 1
 3 1 0 0 1 0 1 1 3 0 1 0 0 0 2 0 3 0 0 3 0 1 0 3 1 1 0 0 0 3 0 0 0 0 3 0 1
 3 0 3 0 0 1 0 1 0 3 1 0 0 0 3 1 3 0 0 3 0 3 3 0 3 3 1 1 0 0 0 0 3 0 0 1 0
 1 3 3 3 0 0 0 3 0 0 0 0 3 0 0 3 0 3 3 0 1 0 3 2 1 3 1 2 3 0 1 0 0 1 1 0 3
 0 0 0 0]


## Save the Best Model

Save your model to the directory that was created for that purpose when mounting the drive.

After the model is saved, it is safe to delete the contents of the directory used for training. You
will need to do this to avoid running out of space.

Check often how much of your google drive quota you are using.

Also keep in mind that items you delete from the file browser on your PC are moved to the
trash, where they remain for 30 days before being deleted. This means that they are still
counted on your quota.

Check the contents of the trash and empty often.

In [ ]:
# save the model
trainer.save_model(models_dir)

In [ ]:
# delete the contents
!rm -r /content/gdrive/MyDrive/slp2-2024-assignment4-checkpoint

## Load saved model
- should now be able to load your model in the same way you loaded the "distilbert-base-uncased" model earlier  
- use AutoModelForSequenceClassification. But this
time you only need to provide the model location,  
- create a TextClassificationPipeline.

In [ ]:
# load the model
model = AutoModelForSequenceClassification.from_pretrained(models_dir)

In [ ]:
# text classification pipelie
from transformers import pipeline

classifier = pipeline(task="text-classification", model=model, tokenizer=tokenizer)

## Use the saved model to make our own inference

Use the pipeline created in the previous step to classify some tweets (at least 15). You can use tweets from the test set for this part.

For each tweet, print the model's prediction, the model's confidence score, and the tweet.

In [ ]:
# select 15 tweets from the test set

tweets = test_data.select(range(15))["text"]

for tweet in tweets:
  result = classifier(tweet)

  #print(result)

  print(f"Prediction: {result[0]['label']}")
  print(f"Confidence Score: {result[0]['score']}\n")
  print(f"Tweet: {tweet}")

Prediction: sadness
Confidence Score: 0.9566965103149414

Tweet: #Deppression is real. Partners w/ #depressed people truly dont understand the depth in which they affect us. Add in #anxiety &amp;makes it worse
Prediction: anger
Confidence Score: 0.8908029794692993

Tweet: @user Interesting choice of words... Are you confirming that governments fund #terrorism? Bit of an open door, but still...
Prediction: sadness
Confidence Score: 0.9643287658691406

Tweet: My visit to hospital for care triggered #trauma from accident 20+yrs ago and image of my dead brother in it. Feeling symptoms of #depression
Prediction: joy
Confidence Score: 0.9395257234573364

Tweet: @user Welcome to #MPSVT! We are delighted to have you! #grateful #MPSVT #relationships
Prediction: joy
Confidence Score: 0.9209134578704834

Tweet: What makes you feel #joyful?
Prediction: anger
Confidence Score: 0.9281604886054993

Tweet: i am revolting.
Prediction: sadness
Confidence Score: 0.9612448811531067

Tweet: Rin might ever 